<a href="https://colab.research.google.com/github/KrishalDhungana/Reptiles-Amphibians-Classifier/blob/main/Gradio_Model_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# IMPORTS
import keras.layers as layers
import matplotlib.pyplot as plt
from keras import Model
from numpy import argmax
from keras.optimizers import SGD, Adam
from pandas import read_csv
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc
from keras.models import Sequential
from keras.utils import to_categorical
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import gradio as gr
import random
import shutil
from keras.applications import EfficientNetB0,MobileNetV2,VGG19
from tensorflow.keras.applications import EfficientNetB0
from zipfile import ZipFile

In [19]:
# Unzip folder
file_name = "new-reptiles-and-amphibians-image-dataset.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done unzipping')

Done unzipping


In [37]:
# CREATE GRADIO INTERFACE FOR IMAGE PREDICTION DEMO

IMAGE_SIZE=(224,224) # images will be resized to this
def build_and_compile_cnn():
    """
    Build and compile a Convolutional Neural Network model.

    The CNN consists of:
    - Three convolutional layers with ReLU activation and max pooling layers.
    - A flattening layer to convert 2D matrices into a 1D vector.
    - A dense (fully connected) layer with ReLU activation.
    - A dense output layer with sigmoid activation for binary classification.

    The model is compiled using the RMSProp optimizer and binary cross-entropy loss.

    Returns:
        model (Sequential): The compiled CNN model.
    """
    # Initialize a sequential model
    model = Sequential([
        # 1: convolutional layer with 32 filters of size 3x3, ReLU activation, and a max pooling layer with window size 2x2
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        # 2: convolutional layer with 64 filters of size 3x3, ReLU activation, and a max pooling layer with window size 2x2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        # 3: convolutional layer with 64 filters of size 3x3, ReLU activation, and a max pooling layer with window size 2x2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        # 4: flatten pixels of resulting image into a 1D vector
        layers.Flatten(),
        # 5: feed result into fully connected dense layer with 64 units and ReLU activation
        layers.Dense(64, activation='relu'),
        # 6. output layer with a single unit and sigmoid activation for binary classification
        layers.Dense(1, activation='sigmoid')
    ])

    # optimizerAdam = Adam(learning_rate=ALPHA) # RMSProp gives much better results
    # Use binary cross-entropy loss function and RMSProp optimizer
    model.compile(loss='binary_crossentropy', optimizer='RMSProp', metrics=['accuracy'])

    return model

# Create a new instance of the raw model by loading saved weights
raw_model = build_and_compile_cnn()
raw_model.load_weights('16_epochs.weights.h5')

IMG_SIZE = 224
NUM_CLASSES=2
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    # prepare EfficientNetB0
    efficientnet = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")
    efficientnet.trainable = False
    x1 = efficientnet.output
    x1 = layers.GlobalAveragePooling2D()(x1)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)

    # prepare MobileNetV2
    mobilenet = MobileNetV2(include_top=False, input_tensor=inputs, weights="imagenet")
    mobilenet.trainable = False
    x2 = mobilenet.output
    x2 = layers.GlobalAveragePooling2D()(x2)
    x2 = layers.BatchNormalization()(x2)
    x2 = layers.Dropout(0.2)(x2)

    # prepare VGG19
    VGG = VGG19(include_top=False, input_tensor=inputs, weights="imagenet")
    VGG.trainable = False
    x3 = mobilenet.output
    x3 = layers.GlobalAveragePooling2D()(x3)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.Dropout(0.2)(x3)

    # Concatenate the 3 layers
    concatenated = layers.concatenate([x1, x2,x3])


    #classification layer
    outputs = layers.Dense(num_classes, activation="softmax")(concatenated)

    # Compile model
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model

# Create a new instance of the ensemble model by loading saved weights
ensemble_model = build_model(NUM_CLASSES)
ensemble_model.load_weights('16_epochs_Ensemble.weights.h5')

# Create df with image paths and labels
image_dir = Path("new-reptiles-and-amphibians-image-dataset")
filepaths = list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
image_df = pd.concat([filepaths, labels], axis=1) # 5994 rows, 2 cols
# Select 10 Reptile and Amphibian images to show in the Gradio interface
def get_sample_image_paths():
    """
    Get sample reptile/amphibian images
    """
    # Select 20 random images (10 Reptile, 10 Amphibian) to show in Gradio
    reptiles = image_df[image_df['Label'] == 'Reptile'].sample(10, random_state=13)
    amphibians = image_df[image_df['Label'] == 'Amphibian'].sample(10, random_state=13)
    sample_df = pd.concat([reptiles, amphibians])
    sample_df = sample_df.sample(frac=1).reset_index(drop=True) # shuffle df
    return sample_df.values.tolist()

# Create function for Gradio interface
def classify_image_raw_model(inp):
    """
    Classify an input image as either 'Amphibian' or 'Reptile' using the raw CNN Model
    """
    # Resize and normalize input image
    inp = inp.resize(IMAGE_SIZE)
    inp = np.array(inp) / 255.0
    inp = inp.reshape((-1, 224, 224, 3))

    # Predict class
    prediction = raw_model.predict(inp)
    predicted_class = int(np.round(prediction[0][0]))

    # Map class index to class name
    class_names = {0: 'Amphibian', 1: 'Reptile'}
    return class_names[predicted_class]

def classify_image_ensemble_model(inp):
    """
    Classify an input image as either 'Amphibian' or 'Reptile' using the ensembled pre-trained CNN model
    """
    # Resize and normalize input image
    inp = inp.resize(IMAGE_SIZE)
    inp = np.array(inp) / 255.0
    inp = inp.reshape((-1, 224, 224, 3)) # For this function, replace any of these to fit the pre-trained model params

    # Predict class
    prediction = ensemble_model.predict(inp) # For this function, replace this with the ensemble model
    predicted_class = int(np.argmax(prediction[0]))

    # Map class index to class name
    class_names = {0: 'Amphibian', 1: 'Reptile'}
    return class_names[predicted_class]

def classify_images(inp, true_label):
    """
    Classify an input image using both models and return both results
    """
    raw_model_pred = classify_image_raw_model(inp)
    ensemble_model_pred = classify_image_ensemble_model(inp)
    return raw_model_pred, ensemble_model_pred, true_label

# Create a Gradio interface
interface = gr.Interface(
    fn=classify_images,
    inputs=[gr.Image(type="pil"), gr.Textbox(label="True Label")],
    outputs=[
        gr.Label(num_top_classes=1, label="Raw Classifier"),
        gr.Label(num_top_classes=1, label="Ensembled Pre-trained Classifier")
    ],
    examples=get_sample_image_paths(), # this gives us images and their true labels
    title="Amphibian vs Reptile Classifier",
    description="Upload an image and specify its true label, or click one of the examples below. Then, click 'Submit' to see whether the models classify it as an Amphibian or a Reptile."
)
interface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
<ipython-input-37-b1861eec70aa>:62: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet = MobileNetV2(include_top=False, input_tensor=inputs, weights="imagenet")
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Sk

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ffa3cd3b41372256fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
